# Figure 7

## Imports

In [1]:
import dolfin  
import matplotlib.pyplot as plt
import numpy
import sympy
import sys

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

## Defining Geometry

In [4]:
seeds_filename = "Fig7-seeds.dat"
mesh_filebasename = "Fig7-mesh"

domain_y = 1.
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])
V0 = (xmax - xmin)*(ymax - ymin)
dV = dolfin.Measure("dx",domain=mesh)
Vs0 = dolfin.assemble(dolfin.Constant(1.) * dV)
Phi_s0 = Vs0/V0

[[0.28867513 0.25      ]
 [0.         0.75      ]
 [0.8660254  0.25      ]
 [0.57735027 0.75      ]]
Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 122 (Line)
Info    : [ 10%] Meshing curve 123 (Line)
Info    : [ 10%] Meshing curve 124 (Line)
Info    : [ 20%] Meshing curve 152 (Line)
Info    : [ 20%] Meshing curve 153 (Line)
Info    : [ 20%] Meshing curve 154 (Line)
Info    : [ 30%] Meshing curve 155 (Line)
Info    : [ 30%] Meshing curve 156 (Line)
Info    : [ 30%] Meshing curve 157 (Line)
Info    : [ 40%] Meshing curve 158 (Line)
Info    : [ 40%] Meshing curve 159 (Line)
Info    : [ 40%] Meshing curve 160 (Line)
Info    : [ 50%] Meshing curve 161 (Line)
Info    : [ 50%] Meshing curve 162 (Line)
Info    : [ 50%] Meshing curve 163 (Line)
Info    : [ 60%] Meshing curve 164 (Line)
Info    : [ 60%] Meshing curve 165 (Line)
Info    : [ 70%] Meshing curve 166 (Line)
Info    : [ 70%] Meshing curve 

/Users/martin/BOULOT/COMPUTATIONAL/Python/micro_poro_structure_generator/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:44: RuntimeWarning: divide by zero encountered in double_scalars
  m = (Q[1] - P[1])/(Q[0] - P[0])
/Users/martin/BOULOT/COMPUTATIONAL/Python/micro_poro_structure_generator/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:61: RuntimeWarning: invalid value encountered in double_scalars
  x = (c2 - c1)/(m1 - m2)
/Users/martin/BOULOT/COMPUTATIONAL/Python/micro_poro_structure_generator/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:62: RuntimeWarning: invalid value encountered in double_scalars
  y = (m2*c1 - m1*c2)/(m2 - m1)
/Users/martin/BOULOT/COMPUTATIONAL/Python/micro_poro_structure_generator/micro_poro_structure_generator/generate_mesh_2D_rectangle_w_voronoi_inclusions.py:45: RuntimeWarning: invalid value encountered in double_scalars
  c = P[1] - m * P[0]


Info    : Running '/opt/miniconda3/envs/fenics-2019-py310/bin/gmsh -2 -o Fig7-mesh.msh -format msh22 Fig7-mesh.msh' [Gmsh 4.11.1, 1 node, max. 1 thread]
Info    : Started on Mon Dec  4 12:03:23 2023
Info    : Reading 'Fig7-mesh.msh'...
Info    : 63 entities
Info    : 781 nodes
Info    : 1280 elements
Info    : Done reading 'Fig7-mesh.msh'
Info    : Meshing 1D...
Info    : Done meshing 1D (Wall 0.0002835s, CPU 0.000255s)
Info    : Meshing 2D...
Info    : Done meshing 2D (Wall 2.1916e-05s, CPU 1.2e-05s)
Info    : 793 nodes 1314 elements
Info    : Writing 'Fig7-mesh.msh'...
Info    : Done writing 'Fig7-mesh.msh'
Info    : Stopped on Mon Dec  4 12:03:23 2023 (From start: Wall 0.00639671s, CPU 0.265442s)
Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Expecting 781 vertices
Found all vertices
Expecting 1280 cells
Found all cells
Conversion done
Phis0=0.2933053485926732
Phif0=0.7066946514073268


## Computing microscopic model response

## Strain loading

### Incompressible

In [3]:
qois_filename = "Fig7-qois.dat"
res_basename = "Fig7"

load_params = {}
load_params["pf"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["U_bar_00"] = 0.5
load_params["U_bar_11"] = 0.5

Es = 1.
nus = 0.499
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_lst_incomp = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
vs_lst_incomp = qois_vals[:, qois_name_list.index("vs") - 1]
Phi_s_lst_incomp = [vs_/vol for vs_ in vs_lst_incomp]

/opt/miniconda3/envs/fenics-2019-py310/lib/python3.10/site-packages/ufl/exproperators.py:336: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if arg in ("+", "-"):


AssertionError: Integration failed. Aborting.

### Compressible

In [ ]:
load_params = {}
load_params["pf"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["U_bar_00"] = 0.5
load_params["U_bar_11"] = 0.5

Es = 1.
nus = 0.2
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_lst_comp = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
vs_lst_comp = qois_vals[:, qois_name_list.index("vs") - 1]
Phi_s_lst_comp = [vs_/vol for vs_ in vs_lst_comp]

In [ ]:
Phi_s_macro_lst = [Phi_s0]*len(eps_lst_incomp) # Macroscopic model response

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x,~E_y~()$', fontsize=16)
plt.ylabel(r'$\Phi_s$', fontsize=16)
plt.plot(eps_lst_comp, Phi_s_lst_comp, '#084594')
plt.plot(eps_lst_incomp, Phi_s_lst_incomp, '#084594', linestyle='dashed')
plt.plot(eps_lst_incomp, Phi_s_macro_lst, '#99000D')
plt.legend(['Microscopic model - compressible solid', 'Microscopic model - quasi-incompressible solid', 'Macroscopic model'])
plt.xlim(0, 0.5)
plt.ylim(0.18, 0.42)
# plt.savefig('Phi-s_E.pdf', bbox_inches='tight') 
plt.show()

## $\Phi_s  -p_f$


### Compressible case

In [ ]:
load_params = {}
load_params["pf"] = 0.5
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["U_bar_00"] = 0.
load_params["U_bar_11"] = 0.

Es = 1.
nus = 0.2
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
pf_lst_comp = qois_vals[:, qois_name_list.index("p_f") - 1]
vs_lst_comp = qois_vals[:, qois_name_list.index("vs") - 1]
Phi_s_lst_comp = [vs_/vol for vs_ in vs_lst_comp]

### Incompressible case

In [ ]:
load_params = {}
load_params["pf"] = 0.5
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["U_bar_00"] = 0.
load_params["U_bar_11"] = 0.

Es = 1
nus = 0.499
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
pf_lst_incomp = qois_vals[:, qois_name_list.index("p_f") - 1]
vs_lst_incomp = qois_vals[:, qois_name_list.index("vs") - 1]
Phi_s_lst_incomp = [vs_/vol for vs_ in vs_lst_incomp]

In [ ]:
Phi_s_macro_lst = [Phi_s0]*len(pf_lst_incomp) # Macroscopic model response

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$p_f~(kPa)$', fontsize=16)
plt.ylabel(r'$\Phi_s$', fontsize=16)
plt.plot(pf_lst_comp, Phi_s_lst_comp, '#084594')
plt.plot(pf_lst_incomp, Phi_s_lst_incomp, '#084594', linestyle='dashed')
plt.plot(pf_lst_incomp, Phi_s_macro_lst, '#99000D')
plt.legend(['Microscopic model - compressible solid', 'Microscopic model - quasi-incompressible solid', 'Macroscopic model'])
plt.xlim(0, 0.5)
plt.ylim(0.18, 0.42)
# plt.savefig('Phi-s_p-f.pdf', bbox_inches='tight') 
plt.show()